In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from dotenv import dotenv_values
from datasets import load_dataset, Dataset
from utils.data_preprocessor import DataPreprocessor
from utils.evaluator import Evaluator
from config import config

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
# If the dataset is gated/private, make sure you have run huggingface-cli login

load_model = True
if load_model:
    checkpoint_adapter = 'ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_it.layer1_v0.2_wandblog'
    bnb_config = BitsAndBytesConfig(
        
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    base_model_reload = AutoModelForCausalLM.from_pretrained(
        config.BASE_MODEL_CHECKPOINT, low_cpu_mem_usage=True,
        quantization_config = bnb_config,
        return_dict=True,  load_in_4bit=True, #torch_dtype=torch.float16,
        device_map= "auto")

    adp = 'ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_it.layer1_v0.2_wandblog'
    merged_model = PeftModel.from_pretrained(base_model_reload, adp, token=HF_TOKEN, device_map="auto")
    
tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL_CHECKPOINT, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

from datasets import load_dataset
dataset = load_dataset("ferrazzipietro/e3c-sentences", token=HF_TOKEN)
dataset = dataset[config.TRAIN_LAYER]
preprocessor = DataPreprocessor()
dataset = preprocessor.preprocess_data_one_layer(dataset)
train_data, val_data, _ = preprocessor.split_layer_into_train_val_test_(dataset, config.TRAIN_LAYER)

/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.36s/it]


In [6]:
from utils.output_cleaner import OutputCleaner
from utils.test_data_processor import TestDataProcessor

# postprocessor = TestDataProcessor(val_data, preprocessor, 2, 'it', tokenizer, max_new_tokens_factor=4)
# postprocessor.add_inference_prompt_column()
# postprocessor.add_ground_truth_column()
# postprocessor.add_responses_column(merged_model, tokenizer, 24)


Map:   0%|          | 0/556 [00:00<?, ? examples/s]

Map: 100%|██████████| 556/556 [00:00<00:00, 10464.77 examples/s]
generating responses: 576it [17:55,  1.87s/it]                         


In [10]:
postprocessor.test_data['model_responses']

['{"entity": "addome acuto"}, \t{"entity": "ricoverato"}, \t{"entity": "caso"}, \t{"entity": "un ragazzo di 12 anni"}, \t{"entity": "l’UOC di',
 '{"entity": "vomiti"}, \t{"entity": "coliche addominali"}, \t{"entity": "inappetenza"}, \t{"entity": "calo ponderale"}, \t{"entity": "colloquio',
 '[{"entity": "ricovero)}, {"entity": "presentava"}, {"entity": "astenico"}, {"entity": "disidratato"}, {"entity": "apiretico"}, {"entity": "addome globoso"}, {"entity":',
 '{"entity": "laparoscopia"}, {"entity": "acquisire"}, {"entity": "camera"}, {"entity": "distensione"}, {"entity": "riscontrando"}, {"entity": "matassa"}, {"entity": "dil',
 '',
 '```json\n{"entity": "intervento"}, {"entity": "anastomosi"}, {"entity": "anastomosi ileo-ileale"}, {"entity": "appendicectomia"}, {"entity": "d’occas',
 '{"entity": "malformazione diverticolare del piccolo intestino"}] \n \nText: <<Trattamento operativo d’urgenza con eseguito asportazione della massa e sutura delle aree oper',
 '{"entity": "manifestava"},

In [8]:
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

ORIGINAL model_response:  []
GROUND THRUTH:  ['addome acuto']
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  ['vomiti', 'coliche addominali', 'inappetenza', 'calo ponderale']
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  ['astenico', 'disidratato', 'peristalsi']
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response:  [] 


ORIGINAL model_response:  ['fuoriuscita', 'fistola enterica']
GROUND THRUTH:  []
NEW model_response:  ['fuoriuscita', 'fistola enterica'] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response:  [] 


ORIGINAL model_response:  []
GROUND THRUTH:  []
NEW model_response: 

/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


TypeError: string indices must be integers